# Réparation des bornes défectueuses dans les extraits

Ce notebook permet d'exécuter interactivement le script de réparation des bornes défectueuses dans les extraits définis dans `extract_sources.json`.

## Contexte

Notre application d'analyse d'argumentation utilise un système d'extraits définis par des marqueurs de début et de fin dans des textes sources. Certains extraits présentent des problèmes de bornes (marqueurs introuvables ou incorrects), notamment dans le corpus de discours d'Hitler qui est particulièrement volumineux.

Ce notebook permet de :

1. Analyser les extraits existants pour détecter les bornes défectueuses
2. Proposer des corrections automatiques pour les bornes défectueuses
3. Valider les corrections proposées
4. Sauvegarder les définitions corrigées
5. Générer un rapport détaillé des modifications

## 1. Importation des modules nécessaires

In [ ]:
import os
import re
import json
import logging
import asyncio
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional, Union
import semantic_kernel as sk
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions.kernel_arguments import KernelArguments

# Ajouter le répertoire parent au chemin d'importation
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("__file__"), '../..')))

# Imports depuis les modules du projet
try:
    from argumentation_analysis.ui.config import ENCRYPTION_KEY, CONFIG_FILE, CONFIG_FILE_JSON
    from argumentation_analysis.ui.utils import load_from_cache, reconstruct_url
    from argumentation_analysis.ui.extract_utils import (
        load_source_text, extract_text_with_markers, find_similar_text,
        load_extract_definitions_safely, save_extract_definitions_safely
    )
    from argumentation_analysis.core.llm_service import create_llm_service
except ImportError:
    # Fallback pour les imports relatifs
    from ...ui.config import ENCRYPTION_KEY, CONFIG_FILE, CONFIG_FILE_JSON
    from ...ui.utils import load_from_cache, reconstruct_url
    from ...ui.extract_utils import (
        load_source_text, extract_text_with_markers, find_similar_text,
        load_extract_definitions_safely, save_extract_definitions_safely
    )
    from ...core.llm_service import create_llm_service

# Import du script de réparation
from repair_extract_markers import (
    ExtractRepairPlugin, setup_agents, analyze_extract,
    repair_extract_markers, generate_report
)

## 2. Configuration du logging

In [ ]:
# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] [%(name)s] %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger("RepairExtractMarkers")

# Création d'un handler pour écrire les logs dans un fichier
file_handler = logging.FileHandler("repair_extract_markers.log")
file_handler.setFormatter(logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'))
logger.addHandler(file_handler)

## 3. Chargement des définitions d'extraits

In [ ]:
# Charger les définitions d'extraits
extract_definitions, error_message = load_extract_definitions_safely(CONFIG_FILE, ENCRYPTION_KEY, CONFIG_FILE_JSON)
if error_message:
    logger.error(f"Erreur lors du chargement des définitions d'extraits: {error_message}")
else:
    logger.info(f"{len(extract_definitions)} sources chargées.")
    
# Afficher un résumé des sources et extraits
print(f"\n=== Résumé des sources et extraits ===")
total_extracts = 0
for i, source in enumerate(extract_definitions):
    source_name = source.get("source_name", f"Source #{i}")
    extracts = source.get("extracts", [])
    total_extracts += len(extracts)
    print(f"Source {i+1}: {source_name} - {len(extracts)} extraits")
print(f"\nTotal: {len(extract_definitions)} sources, {total_extracts} extraits")

## 4. Analyse des extraits existants

In [ ]:
# Fonction pour analyser les extraits existants sans les modifier
def analyze_extracts(extract_definitions):
    results = {
        "valid": [],
        "invalid": []
    }
    
    for source_idx, source_info in enumerate(extract_definitions):
        source_name = source_info.get("source_name", f"Source #{source_idx}")
        print(f"\nAnalyse de la source '{source_name}'...")
        
        # Chargement du texte source
        source_text, url = load_source_text(source_info)
        if not source_text:
            print(f"  ❌ Impossible de charger le texte source: {url}")
            continue
        
        print(f"  ✅ Texte source chargé ({len(source_text)} caractères)")
        
        extracts = source_info.get("extracts", [])
        for extract_idx, extract_info in enumerate(extracts):
            extract_name = extract_info.get("extract_name", f"Extrait #{extract_idx}")
            start_marker = extract_info.get("start_marker", "")
            end_marker = extract_info.get("end_marker", "")
            template_start = extract_info.get("template_start", "")
            
            # Extraction du texte avec les marqueurs actuels
            extracted_text, status, start_found, end_found = extract_text_with_markers(
                source_text, start_marker, end_marker, template_start
            )
            
            # Afficher le résultat
            if start_found and end_found:
                print(f"  ✅ Extrait '{extract_name}' valide")
                results["valid"].append({
                    "source_name": source_name,
                    "extract_name": extract_name,
                    "status": status
                })
            else:
                print(f"  ❌ Extrait '{extract_name}' invalide: {status}")
                results["invalid"].append({
                    "source_name": source_name,
                    "extract_name": extract_name,
                    "status": status,
                    "start_found": start_found,
                    "end_found": end_found
                })
    
    return results

# Analyser les extraits existants
analysis_results = analyze_extracts(extract_definitions)

# Afficher un résumé des résultats
print(f"\n=== Résumé de l'analyse ===")
print(f"Extraits valides: {len(analysis_results['valid'])}")
print(f"Extraits invalides: {len(analysis_results['invalid'])}")

# Afficher les extraits invalides
if analysis_results['invalid']:
    print(f"\n=== Extraits invalides ===")
    for i, invalid in enumerate(analysis_results['invalid']):
        print(f"{i+1}. {invalid['source_name']} -> {invalid['extract_name']}")
        print(f"   Statut: {invalid['status']}")
        print(f"   Marqueur début trouvé: {'Oui' if invalid['start_found'] else 'Non'}")
        print(f"   Marqueur fin trouvé: {'Oui' if invalid['end_found'] else 'Non'}")

## 5. Réparation des bornes défectueuses

In [ ]:
# Créer le service LLM
llm_service = create_llm_service()
if not llm_service:
    logger.error("Impossible de créer le service LLM.")
else:
    logger.info(f"Service LLM créé: {llm_service.service_id}")

In [ ]:
# Option pour filtrer les sources (corpus Hitler uniquement)
filter_hitler_only = False  # Mettre à True pour traiter uniquement le corpus de discours d'Hitler

# Filtrer les sources si l'option est activée
filtered_definitions = extract_definitions
if filter_hitler_only:
    original_count = len(filtered_definitions)
    filtered_definitions = [
        source for source in filtered_definitions 
        if "hitler" in source.get("source_name", "").lower()
    ]
    print(f"Filtrage des sources: {len(filtered_definitions)}/{original_count} sources retenues (corpus Hitler).")

In [ ]:
# Réparer les bornes défectueuses
async def run_repair():
    updated_definitions, results = await repair_extract_markers(filtered_definitions, llm_service)
    return updated_definitions, results

# Exécuter la réparation
updated_definitions, repair_results = await run_repair()

# Afficher un résumé des résultats
status_counts = {"valid": 0, "repaired": 0, "rejected": 0, "unchanged": 0, "error": 0}
for result in repair_results:
    status = result.get("status", "error")
    if status in status_counts:
        status_counts[status] += 1

print(f"\n=== Résumé des réparations ===")
print(f"Total des extraits analysés: {len(repair_results)}")
print(f"Extraits valides: {status_counts['valid']}")
print(f"Extraits réparés: {status_counts['repaired']}")
print(f"Réparations rejetées: {status_counts['rejected']}")
print(f"Extraits inchangés: {status_counts['unchanged']}")
print(f"Erreurs: {status_counts['error']}")

## 6. Génération du rapport

In [ ]:
# Générer le rapport
output_file = "repair_report.html"
generate_report(repair_results, output_file)
print(f"Rapport généré dans '{output_file}'.")

# Afficher le rapport dans le notebook
from IPython.display import IFrame
IFrame(src=output_file, width="100%", height=600)

## 7. Sauvegarde des modifications

In [ ]:
# Option pour sauvegarder les modifications
save_changes = False  # Mettre à True pour sauvegarder les modifications

if save_changes:
    print("Sauvegarde des modifications...")
    success, error_message = save_extract_definitions_safely(
        updated_definitions, CONFIG_FILE, ENCRYPTION_KEY, CONFIG_FILE_JSON
    )
    if success:
        print("✅ Modifications sauvegardées avec succès.")
    else:
        print(f"❌ Erreur lors de la sauvegarde des modifications: {error_message}")
else:
    print("Les modifications n'ont pas été sauvegardées (mettez save_changes=True pour sauvegarder).")

## 8. Analyse des extraits après réparation

In [ ]:
# Analyser les extraits après réparation
if save_changes:
    print("\nAnalyse des extraits après réparation...")
    
    # Recharger les définitions d'extraits
    updated_extract_definitions, error_message = load_extract_definitions_safely(CONFIG_FILE, ENCRYPTION_KEY, CONFIG_FILE_JSON)
    if error_message:
        print(f"Erreur lors du chargement des définitions d'extraits: {error_message}")
    else:
        # Analyser les extraits mis à jour
        updated_analysis_results = analyze_extracts(updated_extract_definitions)
        
        # Afficher un résumé des résultats
        print(f"\n=== Résumé de l'analyse après réparation ===")
        print(f"Extraits valides: {len(updated_analysis_results['valid'])}")
        print(f"Extraits invalides: {len(updated_analysis_results['invalid'])}")
        
        # Comparer avec l'analyse initiale
        print(f"\n=== Comparaison avant/après réparation ===")
        print(f"Extraits valides avant: {len(analysis_results['valid'])} -> après: {len(updated_analysis_results['valid'])}")
        print(f"Extraits invalides avant: {len(analysis_results['invalid'])} -> après: {len(updated_analysis_results['invalid'])}")